In [2]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

2023-06-21 18:26:02.343032: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-

In [11]:
# get data
topic = "roche"

pipeline_executor = PipelineExecutor()
roche = pipeline_executor.execute(query=topic, max_articles=None, overwrite=True)

# safe and load quantum_df using parquet
roche.to_parquet('data/clean/roche.gzip', compression='gzip')
roche = pd.read_parquet('data/clean/roche.gzip')

Getting news article info: 100%|██████████| 588/588 [07:01<00:00,  1.39it/s]


In [ ]:
# Paragraphs - Word - silhouette
wizard = ww.WordWizard(df=roche, interest = 'paragraph')
wizard.create_word_embeddings(wizard.interest) \
    .cluster_embeddings(wizard.interest, algorithm = 'hdbscan') \
    .entitiy_recognition(wizard.interest) \
    .summarize_medoids(wizard.interest) \
    .find_sentiment(wizard.interest) \
    .topic_modelling(wizard.interest)


# Paragraphs - Sentence - silhouette
wizard2 = ww.WordWizard(df=roche, interest = 'paragraph')
wizard2.create_sentence_embeddings(wizard2.interest) \
    .cluster_embeddings(wizard2.interest, algorithm = 'hdbscan') \
    .entitiy_recognition(wizard2.interest) \
    .summarize_medoids(wizard2.interest) \
    .find_sentiment(wizard2.interest) \
    .topic_modelling(wizard2.interest)

In [ ]:
# Article - Word - silhouette
wizard3 = ww.WordWizard(df=roche, interest = 'body')
wizard3.create_word_embeddings(wizard3.interest) \
    .cluster_embeddings(wizard3.interest, algorithm = 'hdbscan') \
    .entitiy_recognition(wizard3.interest) \
    .summarize_medoids(wizard3.interest) \
    .find_sentiment(wizard3.interest) \
    .topic_modelling(wizard3.interest)

# Article - Sentence - silhouette
wizard4 = ww.WordWizard(df=roche, interest = 'body')
wizard4.create_sentence_embeddings(wizard4.interest) \
    .cluster_embeddings(wizard4.interest, algorithm = 'hdbscan') \
    .entitiy_recognition(wizard4.interest) \
    .summarize_medoids(wizard4.interest) \
    .find_sentiment(wizard4.interest) \
    .topic_modelling(wizard4.interest)

In [27]:
# Paragraphs - Word - silhouette
interest = 'paragraph'
wizard5 = ww.WordWizard(df=roche, interest = 'paragraph')
wizard5.create_word_embeddings(wizard5.interest) \
    .cluster_embeddings(wizard5.interest) \
    .entitiy_recognition(wizard5.interest) \
    .summarize_medoids(wizard5.interest) \
    .find_sentiment(wizard5.interest) \
    .topic_modelling(wizard5.interest)


# Paragraphs - Sentence - silhouette
wizard6 = ww.WordWizard(df=roche, interest = 'paragraph')
wizard6.create_sentence_embeddings(wizard6.interest) \
    .cluster_embeddings(wizard6.interest) \
    .entitiy_recognition(wizard6.interest) \
    .summarize_medoids(wizard6.interest) \
    .find_sentiment(wizard6.interest) \
    .topic_modelling(wizard6.interest)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting organizations for column paragraph: 100%|██████████| 3/3 [00:02<00:00,  1.38it/s]
Creating summaries for medoids of column paragraph: 100%|███████

KeyError: 'article'

In [4]:
# Article - Word - silhouette
wizard7 = ww.WordWizard(df=roche, interest = 'body')
wizard7.create_word_embeddings(wizard7.interest) \
    .cluster_embeddings(wizard7.interest) \
    .entitiy_recognition(wizard7.interest) \
    .summarize_medoids(wizard7.interest) \
    .find_sentiment(wizard7.interest) \
    .topic_modelling(wizard7.interest)

# Article - Sentence - silhouette
wizard8 = ww.WordWizard(df=roche, interest = 'body')
wizard8.create_sentence_embeddings(wizard8.interest) \
    .cluster_embeddings(wizard8.interest) \
    .entitiy_recognition(wizard8.interest) \
    .summarize_medoids(wizard8.interest) \
    .find_sentiment(wizard8.interest) \
    .topic_modelling(wizard8.interest)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating summaries for medoids of column body:   0%|          | 0/4 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-pa

KeyboardInterrupt: 

In [25]:
# silhouette score ain't working with too few samples I think
from sklearn.cluster import KMeans
df = wizard.df
from sklearn.metrics import silhouette_score
column = 'paragraph'
k=2
embed_column = column + wizard.EMB_SUFFIX
kk = KMeans(n_clusters=k, n_init='auto').fit(df[embed_column].tolist())
labels = kk.labels_
silhouette_score(df[embed_column].tolist(), labels, metric='euclidean')

0.18544431

In [16]:
# check the number of entities in each cluster
wizard.df['paragraph_sentence_embeddings_clusters'].value_counts()

1    8
0    3
4    2
2    2
7    1
5    1
6    1
3    1
8    1
Name: paragraph_sentence_embeddings_clusters, dtype: int64

In [17]:
# check the number of medoids per cluster
wizard.df[wizard.df['paragraph_sentence_embeddings_clusters_medoids'] == True][['paragraph_sentence_embeddings', 'paragraph_sentence_embeddings_clusters']]

,paragraph_sentence_embeddings,paragraph_sentence_embeddings_clusters
0,"[0.037035826593637466, -0.008866664953529835, ...",0
1,"[-0.029360659420490265, -0.06645254045724869, ...",4
2,"[-0.054638300091028214, -0.03934699669480324, ...",7
4,"[0.021085450425744057, -0.07643812894821167, -...",1
5,"[-0.07643575221300125, -0.07916200160980225, -...",5
6,"[0.047656454145908356, -0.03696030005812645, -...",1
8,"[0.0005669331876561046, -0.07934823632240295, ...",6
9,"[0.059580232948064804, -0.022906964644789696, ...",0
11,"[-0.05660158023238182, 0.0067533645778894424, ...",3
12,"[0.02275889180600643, -0.04268726333975792, -0...",4
